In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Functions used to exercises 1),2),3),4)
#f(,) returns If x is rightly classified by the line defined by points p1 and p2
def f(x,p1,p2):
    w=[p1[0]*p2[1]-p2[0]*p1[1], p1[1]-p2[1] , p1[0]-p2[0]]
    return np.sign(1*w[0]+x[0]*w[1]+x[1]*w[2])
#pla(,) returns If x is rightly classified by the line defined by Its slope w
def pla(x,w):
    return np.sign(1*w[0]+x[0]*w[1]+x[1]*w[2])  

In [3]:
#Exercises 1),2),3),4)
for N in (10,100):
    itera = np.zeros(1000);
    error = np.zeros(1000);
    for run in np.arange(1000):
        #p1 and p1 are the random points to construct f
        p2=np.random.uniform(-1,1,size=2)
        p1=np.random.uniform(-1,1,size=2)

        x = np.random.uniform(-1,1,size=(N,2))
        y = [f(xi,p1,p2) for xi in x]

        w = [0,0,0]
        n = 0
        while n==0:
            
            #Find the indexes of the misclassfied points
            misclass = [];
            for idx,cl in enumerate(y):
                if y[idx] != pla(x[idx],w):
                    misclass.append(idx)
            #Get one random already missclassified point
            idp = misclass[np.random.randint(len(misclass))]

            aug_x = np.append(np.array([[1]]),x[idp]) #It has to match w = [w0,w1,w2] <=> x=[1,x1,x2]
            w = w + y[idp]*aug_x
            itera[run]+=1

            #Check If PLA categorizes all points rightly
            flag=1
            for idx,cl in enumerate(y):
                if y[idx] != pla(x[idx],w):
                    flag=0
            n = flag

        #Calculating P[f!=g]
        x = np.random.uniform(-1,1,size=(50,2)) #Generate 50 random points to check error
        y = [f(xi,p1,p2) for xi in x]
        for idx,cl in enumerate(y):
            if y[idx] != pla(x[idx],w):
                error[run]+=1;
        error[run] = error[run]/50.;
    
    print ("Mean number of iterations for N =",N," (after 1000 runs): ",np.mean(itera)) 
    print ("P[f!=g] for N =",N," (after 1000 runs, with each run sampling 50 points for calculating the error): ",np.mean(error))

Mean number of iterations for N = 10  (after 1000 runs):  9.502
P[f!=g] for N = 10  (after 1000 runs, with each run sampling 50 points for calculating the error):  0.10612000000000002
Mean number of iterations for N = 100  (after 1000 runs):  100.135
P[f!=g] for N = 100  (after 1000 runs, with each run sampling 50 points for calculating the error):  0.013879999999999998


In [4]:
#Exercises 5),6),7)
for N in (10,100):
    E_in = np.zeros(1000);
    E_out = np.zeros(1000);
    itera = np.zeros(1000);
    
    #Repeat the experiment 1000 times
    for run in np.arange(1000):
        p2=np.random.uniform(-1,1,size=2)
        p1=np.random.uniform(-1,1,size=2)

        x = np.random.uniform(-1,1,size=(N,2))
        y = [f(xi,p1,p2) for xi in x]

        x = np.array([np.append(np.array([[1]]),xi) for xi in x])
        w_lin = np.matmul(np.matmul(np.linalg.inv(np.matmul(x.T, x)), x.T),y)
        
        if(N==10):
            w = w_lin
            n = 0
            while n==0:

                #Find the indexes of the misclassfied points
                misclass = [];
                for idx,cl in enumerate(y):
                    if y[idx] != np.sign(np.dot(x[idx],w)):
                        misclass.append(idx)
                if(len(misclass)!=0):        
                    idp = misclass[np.random.randint(len(misclass))]
                    w = w + y[idp]*x[idp]
                    itera[run]+=1

                #Check If PLA categorizes all points rightly
                flag=1
                for idx,cl in enumerate(y):
                    if y[idx] != np.sign(np.dot(x[idx],w)):
                        flag=0
                n = flag
            
            
        if(N==100):
            #Calculating E_in
            for idx,cl in enumerate(y):
                if y[idx] != np.sign(np.dot(w_lin.T,x[idx])):
                    E_in[run] = E_in[run] + 1

            E_in[run] = E_in[run]/N;

            #Calculating E_out
            N_out = 1000
            x = np.random.uniform(-1,1,size=(N_out,2)) #Generate 1000 random points to check error
            y = [f(xi,p1,p2) for xi in x]

            x = np.array([np.append(np.array([[1]]),xi) for xi in x])

            for idx,cl in enumerate(y):
                if y[idx] != np.sign(np.dot(w_lin.T,x[idx])):
                    E_out[run] = E_out[run] + 1

            E_out[run] = E_out[run]/N_out;

    if(N==100):
        print ("For N =",N," (after 1000 runs), E_in: ",np.mean(E_in))
        print ("Using the following of out sample points =",N_out," (after 1000 runs) E_out: ",np.mean(E_out))
    if(N==10):    
        print ("Number of iterations for PLA with Its weights initialized by linear regression", np.mean(itera))

Number of iterations for PLA with Its weights initialized by linear regression 5.049
For N = 100  (after 1000 runs), E_in:  0.03833
Using the following of out sample points = 1000  (after 1000 runs) E_out:  0.04852100000000001


In [5]:
#Exercises 8),9),10)
for exer in (8,9):  
    for N in (1000,):
        E_in = np.zeros(1000);
        E_out = np.zeros(1000);
        itera = np.zeros(1000);
        w = np.zeros(shape=(1000,6));

        #Repeat the experiment 1000 times
        for run in np.arange(1000):

            x = np.random.uniform(-1,1,size=(N,2))
            y = np.array([np.sign(xi[0]*xi[0]+xi[1]*xi[1]-0.6) for xi in x])
            #10% of noise
            k = np.random.choice(len(y), 100, replace=False)
            y[k] = y[k]*(-1)
            
            if(exer == 8):
                x = np.array([np.append(np.array([[1]]),xi) for xi in x])
                w_lin = np.matmul(np.matmul(np.linalg.inv(np.matmul(x.T, x)), x.T),y)
                #Calculating E_in
                for idx,cl in enumerate(y):
                    if y[idx] != np.sign(np.dot(w_lin.T,x[idx])):
                        E_in[run] = E_in[run] + 1

                E_in[run] = E_in[run]/N;
                
            if(exer == 9):
                #Non-linear transformation and application of the standart linear regression. 
                x = np.array([np.concatenate((np.array([1]),xi,np.array([xi[0]*xi[1],xi[0]**2,xi[1]**2])),axis=0) for xi in x])
                w_lin = np.matmul(np.matmul(np.linalg.inv(np.matmul(x.T, x)), x.T),y)
                w[run] = w_lin
                
                #Calculate E_out
                N_out = 1000
                x = np.random.uniform(-1,1,size=(N_out,2)) #Generate 1000 random points to check error
                y = np.array([np.sign(xi[0]*xi[0]+xi[1]*xi[1]-0.6) for xi in x])
                
                #10% of noise
                k = np.random.choice(len(y), 100, replace=False)
                y[k] = y[k]*(-1)

                x = np.array([np.concatenate((np.array([1]),xi,np.array([xi[0]*xi[1],xi[0]**2,xi[1]**2])),axis=0) for xi in x])

                for idx,cl in enumerate(y):
                    if y[idx] != np.sign(np.dot(w_lin.T,x[idx])):
                        E_out[run] = E_out[run] + 1

                E_out[run] = E_out[run]/N_out;
            

    if(exer == 8):
        print ("Exercise 8")
        print ("For N =",N," (after 1000 runs), E_in: ",np.mean(E_in),"\n")
        
    if(exer == 9):
        print ("Exercise 9,10")
        print ("Using the following of out sample points =",N_out," (after 1000 runs) E_out: ",np.mean(E_out))
        print ("W_lin for the non-linear :", np.mean(w,axis=0))

Exercise 8
For N = 1000  (after 1000 runs), E_in:  0.504625 

Exercise 9,10
Using the following of out sample points = 1000  (after 1000 runs) E_out:  0.126301
W_lin for the non-linear : [-9.92194271e-01  2.97683159e-04  2.98937429e-04  3.06372835e-03
  1.55513786e+00  1.55671690e+00]


In [6]:
def E(u,v):
    return (u*np.exp(v)-2*v*np.exp(-u))**2

def gradient(w,E):    
    d = np.double(1)
    #e is how accurate the derivatives are going to be, when they stop changing by values greater than "e" we stop the calculations
    e = np.double(1e-6)
    dex,dey= (np.double(0),np.double(0))
    dex1,dex2,dey1,dey2 = (np.double(0),np.double(1),np.double(0),np.double(1))

    while np.abs(dex1-dex2)>=e:
        dex2 = dex1
        dex1 = (E(w[0]+d,w[1])-E(w[0],w[1]))/d
        dex=dex1
        d = d/2

    d=1    
    while np.abs(dey1-dey2)>=e:
        dey2 = dey1
        dey1 = (E(w[0],w[1]+d)-E(w[0],w[1]))/d
        dey=dey1
        d = d/2

    return np.array([dex,dey])

In [7]:
#Exercises 11,12
n = 0.1;

Ein = np.dtype(np.double)
Ein=100
w = np.array([ 1,  1], dtype=np.double)
e = 1e-13 #Smaller than that respects being near 1e-14
itera = 0;
while Ein >= e:
    itera+=1
    g = (-1)*gradient(w,E)
    #print (g)
    w = w + n*g
    Ein = E(w[0],w[1])
    #print ("Ein: ",Ein, " w:", w)
print ("Exercise 11")
print ("Iterations: ",itera , "\nLast w: ", w, "\nEin :", Ein)

Ein, itera , tog, w = 100, 0, 1, np.array([ 1,  1], dtype=np.double);
for i in np.arange(15):
    itera+=1
    g = (-1)*gradient(w,E)
    if(tog==1):        
        w = w + n*np.array([g[0],0])
        
    if(tog==0):
        w = w + n*np.array([0,g[1]])        
      
    if(tog==1):
        tog=0
    else:
        tog=1
    Ein = E(w[0],w[1])
    #print ("Ein: ",Ein, " w:", w)
    
print ("\nExercise 12")    
print ("Complete iterations: ",itera , "\nFinal error: ", Ein)

Exercise 11
Iterations:  10 
Last w:  [0.04473564 0.02395829] 
Ein : 1.9362996376940967e-14

Exercise 12
Complete iterations:  15 
Final error:  0.25051436277857003


In [8]:
#Exercises 13,14
n=0.01
for N in (100,):
    itera = np.zeros(100);
    error = np.zeros(100);
    
    for run in np.arange(100):
        #p1 and p1 are the random points to construct f
        p2=np.random.uniform(-1,1,size=2)
        p1=np.random.uniform(-1,1,size=2)

        x = np.random.uniform(-1,1,size=(N,2))
        y = [f(xi,p1,p2) for xi in x]
        x = np.array([np.append(np.array([[1]]),xi) for xi in x])
        
        w =  np.array([ 0,  0, 0], dtype=np.double);
        norm = 10
        #Using the stopping rule for convergence indicated:
        while norm>=0.01:
            
            #Permute data X for each epoch inside the Logistic Regression algorithm
            indices_epoch = np.random.permutation(N)
            g,d=np.array([0,0,0]),0
            pre_w = np.copy(w)
            
            #This applies Stochastic Gradient Descent and count the number of epochs via itera[run]
            for idx in indices_epoch:                
                d = 1 + np.exp(y[idx]*np.dot(w.T,x[idx]))
                g = (y[idx]*x[idx])/d; 
                w = w + n*g
            itera[run]+=1
            
            #Recalculate the norm of the previus w with the new value
            norm = np.linalg.norm(pre_w-w);
               
        #Calculating E_out
        x = np.random.uniform(-1,1,size=(50,2)) #Generate 100 random points to check error
        y = [f(xi,p1,p2) for xi in x]
        x = np.array([np.append(np.array([[1]]),xi) for xi in x])
        e = np.zeros(50)
        
        for idx,cl in enumerate(y):
            #Losgistic error
            e[idx] = np.log(1.0 + np.exp(-y[idx]* np.dot(x[idx], w)));
            
        error[run]=np.mean(e)         
    

    print ("E_out for N =",N," (after 100 runs, with each run sampling 50 points for calculating the error): ",np.mean(error))
    print ("Mean number of epochs: ", np.mean(itera))

E_out for N = 100  (after 100 runs, with each run sampling 50 points for calculating the error):  0.10346470812446133
Mean number of epochs:  344.92
